In [1]:
# import packages
import yfinance as yf
import numpy as np
import plotly.graph_objs as go
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# download data
tickers = ['META', 'MSFT', "NVDA", "AAPL", "AMZN"]
data = yf.download(tickers, start = '2019-01-01', end = '2025-01-01', auto_adjust=True)['Close']

# compute asset returns
asset_returns = 100 * np.log(data / data.shift(1)).dropna()

# assign equal weights
n_assets = len(tickers)
weights = np.array([1/n_assets] * n_assets)

# compute portfolio returns
portfolio_returns = asset_returns @ weights

[*********************100%***********************]  5 of 5 completed


In [3]:
pVaR = [0.95, 0.99]
lookback_window = 250

# define test window
day_before_start_index = 949
test_window_start = day_before_start_index + 1
test_window = portfolio_returns.index[test_window_start:]

# define historical VaR function
def historical_var(pnl: np.ndarray, confidence: float) -> float:
    return -np.percentile(pnl, (1 - confidence) * 100)

# simulate P&L and compute VaR over test window
historical_95 = []
historical_99 = []
actual_pnl = []

for i in range(test_window_start, len(portfolio_returns)):
    # get current and previous portfolio values
    prev_value = (1 + portfolio_returns.iloc[i - 1])

    # lookback returns for VaR simulation
    lookback_returns = portfolio_returns.iloc[i - lookback_window:i]

    # simulate P&L using historical returns
    simulated_portfolio_values = prev_value * (1 + lookback_returns)
    simulated_pnl = simulated_portfolio_values - prev_value

    # store actual P&L and VaRs
    actual_pnl.append(portfolio_returns.iloc[i])
    historical_95.append(historical_var(simulated_pnl.values, pVaR[0]))
    historical_99.append(historical_var(simulated_pnl.values, pVaR[1]))

In [4]:
# create results DataFrame
results = pd.DataFrame(index=test_window)
results['ActualPnL'] = actual_pnl
results['Historical95'] = historical_95
results['Historical99'] = historical_99

# VaR violations
results['Violation95'] = results['ActualPnL'] < -results['Historical95']
results['Violation99'] = results['ActualPnL'] < -results['Historical99']

print("95% VaR Violation Rate:", results['Violation95'].mean().round(4) * 100, "%")
print("99% VaR Violation Rate:", results['Violation99'].mean().round(4) * 100, "%")


95% VaR Violation Rate: 7.33 %
99% VaR Violation Rate: 4.47 %


In [5]:
fig = go.Figure()

# actual P&L line
fig.add_trace(go.Scatter(
    x=results.index, y=results['ActualPnL'],
    mode='lines',
    name='Actual P&L',
    line=dict(color='blue')
))

# VaR 95% line
fig.add_trace(go.Scatter(
    x=results.index, y=-results['Historical95'],
    mode='lines',
    name='VaR 95%',
    line=dict(dash='dash', color='orange')
))

# VaR 99% line
fig.add_trace(go.Scatter(
    x=results.index, y=-results['Historical99'],
    mode='lines',
    name='VaR 99%',
    line=dict(dash='dash', color='green')
))

# violation 95% dots
violation95 = results['Violation95']
fig.add_trace(go.Scatter(
    x=results.index[violation95],
    y=results['ActualPnL'][violation95],
    mode='markers',
    name='95% Violation',
    marker=dict(color='red', size=8, symbol='circle'),
    text=[
        f"Date: {d.strftime('%Y-%m-%d')}<br>PnL: {p:.4f}"
        for d, p in zip(results.index[violation95], results['ActualPnL'][violation95])
    ],
    hoverinfo='text'
))

# violation 99% dots
violation99 = results['Violation99']
fig.add_trace(go.Scatter(
    x=results.index[violation99],
    y=results['ActualPnL'][violation99],
    mode='markers',
    name='99% Violation',
    marker=dict(color='black', size=8, symbol='x'),
    text=[
        f"Date: {d.strftime('%Y-%m-%d')}<br>PnL: {p:.4f}"
        for d, p in zip(results.index[violation99], results['ActualPnL'][violation99])
    ],
    hoverinfo='text'
))

# layout adjustments
fig.update_layout(
    title='Historical Simulation VaR with Violations',
    xaxis_title='Date',
    yaxis_title='Portfolio Return / VaR',
    hovermode='closest',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
    height=600
)

fig.show()